In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train=pd.read_csv("../input/spaceship-titanic/train.csv")
train

In [ ]:
test=pd.read_csv("../input/spaceship-titanic/test.csv")
test

In [ ]:
sam=pd.read_csv("../input/spaceship-titanic/sample_submission.csv")
sam

In [ ]:
train.isnull().sum()

# Preprocessing

In [ ]:
train=train.drop(["PassengerId","Cabin","Name"],axis=1)
train

In [ ]:
train.isnull().sum()

# Handling missing variable

In [ ]:
train["HomePlanet"].fillna(train["HomePlanet"].mode()[0],inplace=True)
train["CryoSleep"].fillna(train["CryoSleep"].mode()[0],inplace=True)
train["Destination"].fillna(train["Destination"].mode()[0],inplace=True)
train["VIP"].fillna(train["VIP"].mode()[0],inplace=True)

In [ ]:
train["Age"].fillna(train["Age"].mean(),inplace=True)
train["RoomService"].fillna(train["RoomService"].mean(),inplace=True)
train["FoodCourt"].fillna(train["FoodCourt"].mean(),inplace=True)
train["ShoppingMall"].fillna(train["ShoppingMall"].mean(),inplace=True)
train["Spa"].fillna(train["Spa"].mean(),inplace=True)
train["VRDeck"].fillna(train["VRDeck"].mean(),inplace=True)

In [ ]:
train.isnull().sum()

# Handling Categorical variables

In [ ]:
train=pd.get_dummies(train,columns=["HomePlanet","CryoSleep","Destination","VIP"])

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
train["label"]=le.fit_transform(train["Transported"])

In [ ]:
corr=train.corr()
corr

In [ ]:
train=train.drop(["Transported","HomePlanet_Earth","CryoSleep_False","VIP_False","Destination_TRAPPIST-1e"],axis=1)

In [ ]:
train.describe()

In [ ]:
corr=train.corr()
sns.heatmap(corr)

In [ ]:
corr

In [ ]:
train.describe()

In [ ]:
max_th_VRDeck=train.VRDeck.quantile(0.999)
max_th_Spa=train.Spa.quantile(0.999)
max_th_RoomService=train.RoomService.quantile(0.999)
max_th_FoodCourt=train.FoodCourt.quantile(0.999)
max_th_ShoppingMall=train.ShoppingMall.quantile(0.999)
train_set=train[(train.VRDeck<max_th_VRDeck) & (train.Spa<max_th_Spa) & (train.RoomService<max_th_RoomService) & (train.FoodCourt<max_th_FoodCourt) & (train.ShoppingMall<max_th_ShoppingMall)] 
train_set.shape

In [ ]:
from sklearn.preprocessing import StandardScaler
train_stand=train.copy()
col=['Age','RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']
for i in col:
    stand = StandardScaler().fit(train_stand[[i]])
    train_stand[i] = stand.transform(train_stand[[i]])
train_stand

In [ ]:
#train["Age"]=train["Age"]/25
#train["RoomService"]=train["RoomService"]/200
#train["FoodCourt"]=train["FoodCourt"]/450
#train["ShoppingMall"]=train["ShoppingMall"]/150
#train["Spa"]=train["Spa"]/300
#train["VRDeck"]=train["VRDeck"]/300

# Feature Engineering

# Model Training

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier

In [ ]:
X=train_norm.drop(["label"],axis=1)
y=train_norm["label"]

In [ ]:
from sklearn.model_selection import KFold
kf=KFold(n_splits=100)
kf

In [ ]:
from sklearn.model_selection import StratifiedKFold
skf=StratifiedKFold(n_splits=100)
skf

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,train_size=0.90)
X_train.shape,X_test.shape,y_train.shape,y_test.shape

In [ ]:
def get_score(model,X_train,X_test,y_train,y_test):
    model.fit(X_train,y_train)
    return model.score(X_test,y_test)

In [ ]:
get_score(SVC(kernel='rbf',C=0.9,gamma=0.1),X_train,X_test,y_train,y_test)

In [ ]:
get_score(LogisticRegression(),X_train,X_test,y_train,y_test)

In [ ]:
get_score(RandomForestClassifier(),X_train,X_test,y_train,y_test)

In [ ]:
#from sklearn.model_selection import cross_val_score
#cross_val_score(SVC(),X,y,cv=3,scoring="accuracy").mean()

In [ ]:
#cross_val_score(RandomForestClassifier(),X,y,cv=3,scoring="accuracy").mean()

In [ ]:
#cross_val_score(SVC(kernel='rbf',C=0.9,gamma=0.1),X,y,cv=3,scoring="accuracy").mean()

In [ ]:
#cross_val_score(SVC(kernel='linear',C=10,gamma='auto'),X,y,cv=3,scoring="accuracy").mean()

In [ ]:
#cross_val_score(SVC(kernel='sigmoid',C=10,gamma='auto'),X,y,cv=3,scoring="accuracy").mean()

In [ ]:
#paramgrid={
    
#    'C':[1,10,20],
#    'kernel':['rbf','linear','poly','sigmoid']
    
#}

In [ ]:
#from sklearn.model_selection import RandomizedSearchCV
#clf=RandomizedSearchCV(SVC(gamma='auto'),paramgrid,cv=5,return_train_score=False,n_iter=2)
#clf.fit(X,y)
#pd.DataFrame(clf.cv_results_)[['params_C','params_kernal','mean_test_score']]

In [ ]:
model=SVC(kernel='rbf',C=0.9,gamma=0.1)
model.fit(X,y)

# Preparing test data 

In [ ]:
test

In [ ]:
test=test.drop(["PassengerId","Cabin","Name"],axis=1)

In [ ]:
test.isnull().sum()

In [ ]:
test["HomePlanet"].fillna(test["HomePlanet"].mode()[0],inplace=True)
test["CryoSleep"].fillna(test["CryoSleep"].mode()[0],inplace=True)
test["Destination"].fillna(test["Destination"].mode()[0],inplace=True)
test["VIP"].fillna(test["VIP"].mode()[0],inplace=True)

In [ ]:
test["Age"].fillna(test["Age"].mean(),inplace=True)
test["RoomService"].fillna(test["RoomService"].mean(),inplace=True)
test["FoodCourt"].fillna(test["FoodCourt"].mean(),inplace=True)
test["ShoppingMall"].fillna(test["ShoppingMall"].mean(),inplace=True)
test["Spa"].fillna(test["Spa"].mean(),inplace=True)
test["VRDeck"].fillna(test["VRDeck"].mean(),inplace=True)

In [ ]:
test=pd.get_dummies(test,columns=["HomePlanet","CryoSleep","Destination","VIP"])

In [ ]:
test=test.drop(["HomePlanet_Earth","CryoSleep_False","Destination_55 Cancri e","VIP_False"],axis=1)

In [ ]:
test["Age"]=test["Age"]/25
test["RoomService"]=test["RoomService"]/200
test["FoodCourt"]=test["FoodCourt"]/450
test["ShoppingMall"]=test["ShoppingMall"]/150
test["Spa"]=test["Spa"]/300
test["VRDeck"]=test["VRDeck"]/300

# Predicting

In [ ]:
result=model.predict(test)
result

In [ ]:
label_res=[]
for i in result:
    if(i==1):
        label_res.append("True")
    elif(i==0):
        label_res.append("False")

In [ ]:
label_=pd.DataFrame(label_res,columns=["Transported"])

In [ ]:
sam=sam.drop(["Transported"],axis=1)

In [ ]:
submission=pd.concat([sam,label_],axis=1)
submission

In [ ]:
submission.to_csv("submission.csv",index=False)

In [ ]:
!kaggle competitions submit -c spaceship-titanic -f submission.csv -m "Message"